In [1]:
import os
os.environ["HTTPS_PROXY"] = ""
os.environ["HTTP_PROXY"] = ""

In [3]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

import warnings
warnings.filterwarnings('ignore')

### List some samples for quick review

##### 10725

In [25]:
from dsgtools import azure
venmo = pd.read_adls("Analytics/Customer Projects/Synchrony Financial - Proj 10725 (Venmo)/File from the Customer/New File with DOB/Updated Performance/FINAL_DATA_SHARE_WITH_NEWPERF.csv", 
                     reader = pd.read_csv, encoding='iso-8859-1', delimiter = "|")
full_ssn = pd.read_adls("Analytics/Customer Projects/Synchrony Financial - Proj 10725 (Venmo)/File from the Customer/PPC_CoBrand_FullSSN/Final_data_share_ppc_cob_dobssn.csv.gz", 
                        reader = pd.read_csv, delimiter = "|", compression = "gzip", encoding='iso-8859-1', verbose = False, low_memory = False)
snn4 = pd.read_adls("Analytics/Customer Projects/Synchrony Financial - Proj 10725 (Venmo)/File from the Customer/PPC_CoBrand_Last4SSN/Final_data_share_ppc_cob_dobssn4.csv.gz", 
                    reader = pd.read_csv, delimiter = "|", compression = "gzip", encoding='iso-8859-1', verbose = False, low_memory = False)

print(venmo.shape)
print(full_ssn.shape)
print(snn4.shape)

(1142813, 45)
(933600, 31)
(578087, 31)


In [17]:
venmo["date"] = venmo["application_date_time"].str.slice(2, 7)
venmo = venmo[(venmo["application_outcome"] == "Approved")]
freq(venmo.writeoff_count, venmo.DUE_90P_ACCTS, cross = False, observed = True)

Count       Pct  Cuml Count  Cuml Pct
writeoff_count DUE_90P_ACCTS                                          
0.0            0.0            362404.0  0.980942    362404.0  0.980942
               1.0              3795.0  0.010272    366199.0  0.991214
1.0            0.0              1873.0  0.005070    368072.0  0.996284
               1.0              1313.0  0.003554    369385.0  0.999838
Missing        Missing            60.0  0.000162    369445.0  1.000000

In [18]:
venmo = venmo[(venmo["application_outcome"] == "Approved") & (venmo.writeoff_count.notnull()) & (venmo.DUE_90P_ACCTS.notnull()) & (venmo.fraud_woff_count ==0)]
venmo["tag"] = np.where(((venmo.writeoff_count ==1) | (venmo.DUE_90P_ACCTS ==1)), 1, 0)
bivariate(venmo["date"], venmo["tag"])

tag           tag                                                
stats           N      PctN     Sum      Mean       WoE        IV
date                                                             
APR21     46501.0  0.126120   886.0  0.019053  0.111016  0.001641
AUG21     28778.0  0.078052   170.0  0.005907 -1.073353  0.055764
DEC20     27754.0  0.075275   649.0  0.023384  0.320250  0.009041
FEB21     46537.0  0.126218   971.0  0.020865  0.203700  0.005786
JAN21     31201.0  0.084624   626.0  0.020063  0.163703  0.002457
JUL21     36067.0  0.097821   370.0  0.010259 -0.517028  0.020560
JUN21     40672.0  0.110311   501.0  0.012318 -0.332004  0.010397
MAR21     44427.0  0.120495   828.0  0.018637  0.088514  0.000986
MAY21     47754.0  0.129519   810.0  0.016962 -0.007386  0.000007
NOV20     16621.0  0.045080   420.0  0.025269  0.399717  0.008781
OCT20      2391.0  0.006485    68.0  0.028440  0.521184  0.002285
Missing       0.0  0.000000     0.0  0.000000  0.000000  0.000000
Total    368703.0  1.000000  6299.0  0.017084  0.000000  0.117704

In [27]:
full_ssn["date"] = full_ssn["application_date"].str.slice(0, 7)
full_ssn = full_ssn[(full_ssn["application_outcome"] == "A")]
freq(full_ssn.writeoff, full_ssn.DUE_90P_ACCTS, cross = False, observed = True)

Count       Pct  Cuml Count  Cuml Pct
writeoff DUE_90P_ACCTS                                          
0.0      0.0            655274.0  0.964807    655274.0  0.964807
         1.0              4166.0  0.006134    659440.0  0.970941
1.0      0.0             17515.0  0.025789    676955.0  0.996730
         1.0              1678.0  0.002471    678633.0  0.999201
Missing  Missing           543.0  0.000799    679176.0  1.000000

In [31]:
full_ssn = full_ssn[(full_ssn["application_outcome"] == "A") & (full_ssn.writeoff.notnull()) & (full_ssn.DUE_90P_ACCTS.notnull()) & (full_ssn.fraud_woff_count ==0)]
full_ssn["tag"] = np.where(((full_ssn.writeoff ==1) | (full_ssn.DUE_90P_ACCTS ==1)), 1, 0)
bivariate(full_ssn["date"], full_ssn["tag"])

tag           tag                                                 
stats           N      PctN      Sum      Mean       WoE        IV
date                                                              
2020-01   31077.0  0.046360   2177.0  0.070052  1.186222  0.119004
2020-02   45468.0  0.067828   3028.0  0.066596  1.131927  0.154073
2020-03   35556.0  0.053041   2232.0  0.062774  1.068736  0.103902
2020-04   25739.0  0.038397   1063.0  0.041299  0.627380  0.020618
2020-05   29273.0  0.043668   1071.0  0.036587  0.501316  0.014044
2020-06   26472.0  0.039490    933.0  0.035245  0.462559  0.010603
2020-07   22775.0  0.033975    758.0  0.033282  0.403229  0.006729
2020-08   13681.0  0.020409    412.0  0.030115  0.299954  0.002125
2020-09   11685.0  0.017431    370.0  0.031665  0.351734  0.002560
2020-10   10792.0  0.016099    260.0  0.024092  0.070624  0.000083
2020-11   11160.0  0.016648    253.0  0.022670  0.008345  0.000001
2020-12    9971.0  0.014874    199.0  0.019958 -0.121856  0.000208
2021-01   11256.0  0.016791    189.0  0.016791 -0.297860  0.001296
2021-02   10378.0  0.015482    189.0  0.018212 -0.215201  0.000648
2021-03   11996.0  0.017895    161.0  0.013421 -0.525296  0.003877
2021-04   11897.0  0.017748    184.0  0.015466 -0.381403  0.002162
2021-05   12592.0  0.018784    249.0  0.019774 -0.131275  0.000304
2021-06   17130.0  0.025554    236.0  0.013777 -0.498766  0.005050
2021-07   59877.0  0.089322    782.0  0.013060 -0.552931  0.021180
2021-08   66640.0  0.099411    308.0  0.004622 -1.600212  0.129288
2021-09   58002.0  0.086525     12.0  0.000207 -4.711003  0.413160
2021-10   63872.0  0.095282      6.0  0.000094 -5.500667  0.533930
2021-11   73058.0  0.108985      1.0  0.000014 -7.426879  0.827535
Missing       0.0  0.000000      0.0  0.000000  0.000000  0.000000
Total    670347.0  1.000000  15073.0  0.022485  0.000000  2.372380

In [32]:
snn4["date"] = snn4["application_date"].str.slice(0, 7)
snn4 = snn4[(snn4["application_outcome"] == "A")]
freq(snn4.writeoff, snn4.DUE_90P_ACCTS, cross = False, observed = True)

,,Count,Pct,Cuml Count,Cuml Pct
writeoff,DUE_90P_ACCTS,,,,
Missing,Missing,1.0,1.0,1.0,1.0


In [ ]:
full_ssn = full_ssn[(full_ssn["application_outcome"] == "A") & (full_ssn.writeoff.notnull()) & (full_ssn.DUE_90P_ACCTS.notnull()) & (full_ssn.fraud_woff_count ==0)]
full_ssn["tag"] = np.where(((full_ssn.writeoff ==1) | (full_ssn.DUE_90P_ACCTS ==1)), 1, 0)
bivariate(full_ssn["date"], full_ssn["tag"])

tag           tag                                                 
stats           N      PctN      Sum      Mean       WoE        IV
date                                                              
2020-01   31077.0  0.046360   2177.0  0.070052  1.186222  0.119004
2020-02   45468.0  0.067828   3028.0  0.066596  1.131927  0.154073
2020-03   35556.0  0.053041   2232.0  0.062774  1.068736  0.103902
2020-04   25739.0  0.038397   1063.0  0.041299  0.627380  0.020618
2020-05   29273.0  0.043668   1071.0  0.036587  0.501316  0.014044
2020-06   26472.0  0.039490    933.0  0.035245  0.462559  0.010603
2020-07   22775.0  0.033975    758.0  0.033282  0.403229  0.006729
2020-08   13681.0  0.020409    412.0  0.030115  0.299954  0.002125
2020-09   11685.0  0.017431    370.0  0.031665  0.351734  0.002560
2020-10   10792.0  0.016099    260.0  0.024092  0.070624  0.000083
2020-11   11160.0  0.016648    253.0  0.022670  0.008345  0.000001
2020-12    9971.0  0.014874    199.0  0.019958 -0.121856  0.000208
2021-01   11256.0  0.016791    189.0  0.016791 -0.297860  0.001296
2021-02   10378.0  0.015482    189.0  0.018212 -0.215201  0.000648
2021-03   11996.0  0.017895    161.0  0.013421 -0.525296  0.003877
2021-04   11897.0  0.017748    184.0  0.015466 -0.381403  0.002162
2021-05   12592.0  0.018784    249.0  0.019774 -0.131275  0.000304
2021-06   17130.0  0.025554    236.0  0.013777 -0.498766  0.005050
2021-07   59877.0  0.089322    782.0  0.013060 -0.552931  0.021180
2021-08   66640.0  0.099411    308.0  0.004622 -1.600212  0.129288
2021-09   58002.0  0.086525     12.0  0.000207 -4.711003  0.413160
2021-10   63872.0  0.095282      6.0  0.000094 -5.500667  0.533930
2021-11   73058.0  0.108985      1.0  0.000014 -7.426879  0.827535
Missing       0.0  0.000000      0.0  0.000000  0.000000  0.000000
Total    670347.0  1.000000  15073.0  0.022485  0.000000  2.372380

In [28]:
full_ssn["fraud_woff_count"].value_counts()

0    670890
1      8286
Name: fraud_woff_count, dtype: int64

##### 11103

In [50]:
path = "Analytics/Customer Projects/Extra-proj 11103 (CIID FI)/File from the Customer/"
label = pd.read_adls(path + "labeled_data.csv")
input_df = pd.read_adls(path + "lexus_nexis_test_file_distinct_w_ssn.csv", verbose = False, low_memory = False)

print(label.shape)
print(input_df.shape)

input_df = input_df.drop_duplicates(subset = ["UNIQUE_MEMBER_ID"])
label = label.drop_duplicates(subset = ["UNIQUE_MEMBER_ID"])
print(label.shape)
print(input_df.shape)

(100000, 4)
(100000, 25)
(99988, 4)
(100000, 25)


In [51]:
input_df = input_df.merge(label, on = "UNIQUE_MEMBER_ID")
print(input_df.shape)

(99870, 28)


In [52]:
input_df = input_df[input_df.USE_FLAG == 1]
print(input_df.shape)

(56177, 28)


In [54]:
bivariate(input_df.APPLICATION_DATE.str.slice(0, 7), input_df.D90_FLAG)

tag              D90_FLAG                                                
stats                   N      PctN     Sum      Mean       WoE        IV
APPLICATION_DATE                                                         
2021-10              68.0  0.001210    12.0  0.176471  0.486288  0.000344
2021-11            2782.0  0.049522   526.0  0.189073  0.570686  0.019958
2021-12            9420.0  0.167684  1191.0  0.126433  0.093862  0.001531
2022-01           23887.0  0.425210  2498.0  0.104576 -0.120653  0.005909
2022-02           20020.0  0.356374  2313.0  0.115534 -0.008682  0.000027
Missing               0.0  0.000000     0.0  0.000000  0.000000  0.000000
Total             56177.0  1.000000  6540.0  0.116418  0.000000  0.027769